# Deep Q-Learning

## import module

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

## set parameters

In [13]:
batch_size = 32
lr =0.01
EPSILON = 0.9  #用于控制探索和根据经验运动的比例
GAMMA = 0.9  
TARGET_REPLACE_ITER =100   #用于控制多久更新一次target网络
MEMORY_CAPACITY = 2000  #记忆池的大小
env = gym.make('CartPole-v0')  #立木杆游戏
env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape
# isinstance 用于确定env.action是否是int类型-即是否是可使用的动作， 如果是则ENV_A_SHAPE返回0， 否则则返回动作空间的shape

## Build NN Model

In [14]:
class Net(nn.Module):
    def __init__(self, ):
        #注意，这里的输入是状态的维度，输出是动作空间的维度
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization 设置一个随机初始状态
        self.out = nn.Linear(50, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

## Build DQN

### Main part: choose action, store transition, learn from data

In [24]:
class DQN(object):
    #init 这部分注意进行超参数的初始化和设置内置NN的一些超参数
    def __init__(self):#实例化两个网络用于评估
        
        self.eval_net, self.target_net = Net(), Net()
        
        #设置一个变量用于更新target网络
        self.learn_step_counter = 0                                     # for target updating

        #设置两个变量用于存储memory及其计数器
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        
        #此处的N_STATES*2+2是因为：memory的每一行会存：1个state，一个action， 一个reward，和一个next state
        #共计这么多个数据，放置在同一维中
        
        #设置内置的NN的超参数：优化器和损失函数
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=lr)
        self.loss_func = nn.MSELoss()

        
    def choose_action(self, x):
        #选择动作的函数
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy #随机生成一个数，看看是不是小于Epsilon，这样就能概率性地进入搜索和读取
            actions_value = self.eval_net.forward(x) #使用eval网络进行动作价值的评估
            action = torch.max(actions_value, 1)[1].data.numpy() #选择动作中价值最大的那一个 ，1代表取动作的index而不是值，.data用于仅取值，放弃其他属性，.numppy用于转换成数组
            #接下来，如果ENV_A_SHAPE==0，说明action是可以使用的动作，则action取值为action[0]
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random  #随机探索空间，提升经验
            #动作由（0，N_ACTIONS）范围内随机生成一个整数来确定
            action = np.random.randint(0, N_ACTIONS)
            #如果这个action是可用的，则返回action，否则将action重shape成范围内的数
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action  #返回动作

    
    def store_transition(self, s, a, r, s_): 
        #存储记忆，从此前的一些经历获得记忆
        transition = np.hstack((s, [a, r], s_)) #transition由状态s，动作a，奖励r和s_组成
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY #设置该传入记忆transition的index
        self.memory[index, :] = transition #将index放入memory 
        #此处的transition是一个一维的列表[0:N_STATES,1,1,0:N_STATES]的数据
        self.memory_counter += 1 #增加memory计数

        
    def learn(self):
        #使用learn进行target NN网络的更新和memory等内容的输入/输出
        
        # target parameter update
        #如果学校补偿除以target网络更新步数为0，即为整数倍66 
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            #目标网络使用eval网络进行更新-此处：load_state_dict是使用括号内的state_dict()来更新权重
            #state_dict()是读取网络的权重
            self.target_net.load_state_dict(self.eval_net.state_dict())
        
        #学习过程进入一次，增加一次学习计数
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, batch_size) #从memory中随机选取batch_size个index
        b_memory = self.memory[sample_index, :] #从memory中的对应index获得些数据
        #b_memory [state,action, reward, state_next]的一个一维数据
        b_s = torch.FloatTensor(b_memory[:, :N_STATES]) #取一维数据的前N_STATES，为状态
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)) #接下来一个为动作的代表值
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])#然后是奖励 
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:]) #最后是下一个状态
        #以上四个最终给出的都是一个batch_index✖本身数据长度的tensor

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1) 
        #从eval_net中输入当前状态获得网络输出的动作值的列表，有多少个动作就有多少个值的列表
        #使用gather将eval_net的输出沿着dim=1的轴，根据b_a也就是batch_index×1动作列表来进行整合
        #最终输出的q_eval是一个（batch，1）的量，内容是得出的eval网络的对应动作在batch上的估计值
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        #而使用target网络来获取一个b_s_对应的下一个状态的各个动作的评价值，由于当前循环不更新target_net，所以说采用.detach放弃方向传播
        #q_next的shape是batch✖actionnumber
        
        q_target = b_r + GAMMA * q_next.max(1)[0].view(batch_size, 1)   # shape (batch, 1)
        #设计目标函数为： reward加上学习率✖[batch✖最大价值动作] 整个batch内的都获得了新的q_target
        
        #常规4件套
        loss = self.loss_func(q_eval, q_target) 
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

## Instance

In [ ]:
dqn = DQN() #实例化DQN网络
print('Collecting Experience')
#获取经验以训练网络
for i_episode in range(400): #训练400次的i_episode以获取经验
    s = env.reset() #将竖条设置为初始状态，env.reset()函数用于重置环境，该函数将使得环境的initial observation重置
    ep_r = 0 #设置reward
    while True:
        env.render() #env.render()函数用于渲染出当前的智能体以及环境的状态
        a = dqn.choose_action(s) #选择一个动作来改变当前状态

        # take action
        s_, r, done, info = env.step(a) #采取动作，获得下一个状态s_，奖励r和是否应该reset环境，也就是是否完成任务
        #env.step()会返回四个值：observation（object）、reward（float）、done（boolean）、info（dict）

        # modify the reward
        x, x_dot, theta, theta_dot = s_  #将s_这一状态(object形态)给到几个变量
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8 #
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)  #将状态，动作，奖励和状态s_保存到memory里面

        ep_r += r  #获得总的奖励值
        if dqn.memory_counter > MEMORY_CAPACITY: #如果超出存储范围
            dqn.learn()  #更新target网络
            if done:  
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))  #打印总的迭代数和总奖励值

        if done:
            break
        s = s_  #将下一状态s_赋值到状态s

Ep:  202 | Ep_r:  2.52
Ep:  203 | Ep_r:  0.96
Ep:  204 | Ep_r:  3.13
Ep:  205 | Ep_r:  1.57
Ep:  206 | Ep_r:  2.9
Ep:  207 | Ep_r:  3.83
Ep:  208 | Ep_r:  3.22
Ep:  209 | Ep_r:  2.87
Ep:  210 | Ep_r:  1.38
Ep:  211 | Ep_r:  1.81
Ep:  212 | Ep_r:  2.74
Ep:  213 | Ep_r:  3.27
Ep:  214 | Ep_r:  -0.45
Ep:  215 | Ep_r:  2.97
Ep:  216 | Ep_r:  19.11
Ep:  217 | Ep_r:  2.8
Ep:  218 | Ep_r:  0.06
Ep:  219 | Ep_r:  25.75
Ep:  220 | Ep_r:  3.7
Ep:  221 | Ep_r:  82.28
Ep:  222 | Ep_r:  52.02
Ep:  223 | Ep_r:  31.81
Ep:  224 | Ep_r:  4.13
Ep:  225 | Ep_r:  10.22
Ep:  226 | Ep_r:  12.2
Ep:  227 | Ep_r:  10.26
Ep:  228 | Ep_r:  30.58
Ep:  229 | Ep_r:  40.92
Ep:  230 | Ep_r:  32.58
Ep:  231 | Ep_r:  76.95
Ep:  232 | Ep_r:  107.03
Ep:  233 | Ep_r:  38.13
Ep:  234 | Ep_r:  109.76
Ep:  235 | Ep_r:  591.97
Ep:  236 | Ep_r:  172.99
Ep:  237 | Ep_r:  203.75
Ep:  238 | Ep_r:  414.54
Ep:  239 | Ep_r:  332.68
Ep:  240 | Ep_r:  460.71
